In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract


img_board = cv2.imread('assets/5x5.jpg')
img_board = cv2.rotate(img_board, cv2.ROTATE_90_COUNTERCLOCKWISE)
img_board_gray = cv2.cvtColor(img_board, cv2.COLOR_BGR2GRAY)
img_h,img_w = img_board_gray.shape
background_thresh = img_board_gray[0][0]
ADD_THRESH = 90
blur = cv2.GaussianBlur(img_board_gray,(5,5),0)
total_thresh = background_thresh + ADD_THRESH
_,thresh_img = cv2.threshold(blur,total_thresh,255,cv2.THRESH_BINARY)
contours, hier = cv2.findContours(thresh_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
top_25_contours = sorted(contours, key=lambda x : cv2.contourArea(x) if cv2.contourArea(x) < (img_h * img_w)/25 else 0,reverse=True)[:25]

# sort x and y later
coords_and_index = []
for i,contour in enumerate(top_25_contours):
    x, y, _, _ = cv2.boundingRect(contour)
    coords_and_index.append((i,x,y))
print(coords_and_index)
sorted_y = sorted(coords_and_index,key=lambda x:x[2])
print(sorted_y)
for i in range(5):
    sorted_y[5 * i:5* (i + 1)] = sorted(sorted_y[5 * i:5* (i + 1)], key=lambda x:x[1])
top_25_sorted = [top_25_contours[i[0]] for i in sorted_y]
print(sorted_y)


[(0, 206, 2360), (1, 3231, 2308), (2, 1695, 2286), (3, 952, 1829), (4, 956, 2341), (5, 2479, 2279), (6, 199, 1841), (7, 2472, 1787), (8, 3201, 1791), (9, 1705, 1790), (10, 205, 1313), (11, 946, 1277), (12, 2491, 1263), (13, 3208, 1251), (14, 970, 734), (15, 236, 758), (16, 2502, 701), (17, 3201, 694), (18, 218, 196), (19, 1702, 715), (20, 1733, 1260), (21, 1701, 183), (22, 3211, 171), (23, 966, 192), (24, 2439, 159)]
[(24, 2439, 159), (22, 3211, 171), (21, 1701, 183), (23, 966, 192), (18, 218, 196), (17, 3201, 694), (16, 2502, 701), (19, 1702, 715), (14, 970, 734), (15, 236, 758), (13, 3208, 1251), (20, 1733, 1260), (12, 2491, 1263), (11, 946, 1277), (10, 205, 1313), (7, 2472, 1787), (9, 1705, 1790), (8, 3201, 1791), (3, 952, 1829), (6, 199, 1841), (5, 2479, 2279), (2, 1695, 2286), (1, 3231, 2308), (4, 956, 2341), (0, 206, 2360)]
[(18, 218, 196), (23, 966, 192), (21, 1701, 183), (24, 2439, 159), (22, 3211, 171), (15, 236, 758), (14, 970, 734), (19, 1702, 715), (16, 2502, 701), (17, 320

In [2]:
print_img = cv2.cvtColor(thresh_img, cv2.COLOR_BGR2RGB)
cv2.drawContours(print_img, top_25_contours, -1, (0, 255, 0), 3)

font = cv2.FONT_HERSHEY_SIMPLEX

fontScale = 10
color = (255, 0, 0)
thickness = 5
for i, place in enumerate(sorted_y):  
    cv2.putText(print_img, str(i), (place[1] + 10,place[2] + 10), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
imS = cv2.resize(print_img, (960, 540)) 
cv2.imshow('Contours', imS)
cv2.waitKey(0)

-1

In [3]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]

    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

        

    return warp

In [25]:
def find_words(top_25_sorted):
    words = []
    for cont in top_25_sorted:
        peri = cv2.arcLength(cont,True)
        approx = cv2.approxPolyDP(cont,0.01*peri,True)
        pts = np.float32(approx)
        corner_pts = pts

        x,y,w,h = cv2.boundingRect(cont)
        width, height = w, h

        average = np.sum(pts, axis=0)/len(pts)
        cent_x = int(average[0][0])
        cent_y = int(average[0][1])
        center = [cent_x, cent_y]

        warp = cv2.rotate(flattener(img_board, pts, w, h),cv2.ROTATE_90_COUNTERCLOCKWISE)
        cropped_img = warp[warp.shape[0]//2:][:]
        blur = cv2.GaussianBlur(cropped_img, (3,3), 0)
        contrast = cv2.convertScaleAbs(blur, alpha=2, beta=0)
        thresh = cv2.threshold(contrast, 0, 255, cv2.THRESH_BINARY_INV  + cv2.THRESH_OTSU)[1]
        words.append(pytesseract.image_to_string(thresh, lang='eng', config='--psm 6').strip())
    return words


In [26]:
find_words(top_25_sorted)

['SOUL',
 'HAM',
 'SPRING',
 'SUB',
 'STOCK',
 'SEAL',
 'NUT',
 'HIMALAYAS',
 'TAP',
 'FILM',
 'POINT',
 'GLOVE',
 'WAKE',
 'SHARK',
 'CAST',
 'DOCTOR',
 'JAM',
 'CAR',
 'DIAMOND',
 'NURSE',
 'PIE',
 'PIANO',
 'ROBOT',
 'WASHER',
 'LAB']